# Binary Classification Sonar Project 1 for the Navy:

# Mines vs. Rocks

# By Sequential API

We will start off by importing all of the classes and functions we will need:

In [8]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Next, we can initialize the random number generator to ensure that we always get the same results when executing this code. This will help if we are debugging:

In [9]:
seed = 7      # fix random seed for reproducibility

In [10]:
np.random.seed(seed)

Now we can load the dataset using pandas and split the columns into 60 input variables (X) and 1 output variable (Y). We use pandas to load the data because it easily handles strings (the output variable), whereas attempting to load the data directly using NumPy would be more difficult.

In [11]:
dataframe = pd.read_csv('sonar.csv', header = None)

In [12]:
dataset = dataframe.values

In [13]:
X = dataset[:,0:60].astype('float')

In [14]:
Y = dataset[:,60]

In [15]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables(one hot encoded)

# Baseline Model

In [16]:
#baseline model
def create_baseline():
    # create model, write code below
    model = models.Sequential()
    model.add(layers.Dense(60, activation = 'relu', input_shape = (60,)))
    model.add(layers.Dense(1, activation = 'sigmoid'))
    
    # Compile model, write code below
    model.compile(optimizer = 'Adam',
                 loss = 'binary_crossentropy',
                 metrics = ['accuracy'])
    
    return model

In [17]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=101, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Baseline: 83.64% (4.47%)


# Baseline Model With Standardized Dataset

In [18]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 85.59% (7.46%)


# Smaller Model

In [19]:
#smaller model
def create_smaller():
    # create model
    model = Sequential()
    model.add(Dense(30, activation = 'relu', input_shape = (60,)))
    model.add(Dense(1, activation = 'sigmoid'))

    # Compile model
    model.compile(optimizer = 'Adam',
                 loss = 'binary_crossentropy',
                 metrics = ['accuracy'])
    
    return model

In [20]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 83.09% (6.41%)


# Large Model

In [21]:
#larger model
def create_larger():
    # create model
    model = Sequential()
    model.add(Dense(60, activation = 'relu', input_shape = (60,)))
    model.add(Dense(30, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    
    # Compile model
    model.compile(optimizer = 'Adam',
                 loss = 'binary_crossentropy',
                 metrics = ['accuracy'])
    return model

In [22]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 85.09% (7.25%)


# Overfit Model

In [23]:
#overfit model
def overfit_model():
    # create model
    model = Sequential()
    model.add(Dense(100, activation = 'relu', input_shape = (60,)))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    
    # Compile model
    model.compile(optimizer = 'Adam',
                 loss = 'binary_crossentropy',
                 metrics = ['accuracy'])
    return model

In [24]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=overfit_model, epochs=300, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Overfit: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Overfit: 84.54% (7.52%)


# Without Using Scikit-Learn

# K-Fold Cross Validation

In [25]:
kf = 15
num_val_samples = len(X) // kf
num_epochs = 100
all_scores = []

In [26]:
for i in range(kf):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [X[:i * num_val_samples],
        X[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [encoded_Y[:i * num_val_samples],
        encoded_Y[(i + 1) * num_val_samples:]],
        axis=0)
    model = create_baseline()
    model.fit(partial_train_data, partial_train_targets,
        epochs=num_epochs, batch_size=5, verbose=0)
    val_err, val_acc = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_acc)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9
processing fold # 10
processing fold # 11
processing fold # 12
processing fold # 13
processing fold # 14


In [27]:
all_scores

[0.3076923191547394,
 0.692307710647583,
 0.6153846383094788,
 0.5384615659713745,
 0.7692307829856873,
 0.7692307829856873,
 0.23076923191547394,
 0.3076923191547394,
 0.23076923191547394,
 0.38461539149284363,
 0.6153846383094788,
 0.6153846383094788,
 0.4615384638309479,
 0.6153846383094788,
 1.0]

In [28]:
###### all_scores
print("K-Fold: %.2f%% (%.2f%%)" % (np.mean(all_scores)*100, np.std(all_scores)*100))

K-Fold: 54.36% (21.48%)
